# Notebook for drawing nice pictures for posts or presentations

In this notebook, we reproduce nice plottings to be used in linkedin posts and presentations.
In particular, the following images can be reproduced:
- Image of the network and the population obtained in the gadm shapes 

## Parameters and imports
### Paths

In [ ]:
# Paths of build shapes
# path of the country shapes
path_country_shapes = 'resources/country_shapes.geojson'
# path of the off shore shapes
path_off_shore_shapes = 'resources/offshore_shapes.geojson'
# path of the gadm shapes
path_gadm_shapes = 'resources/gadm_shapes.geojson'

# Path of the OSM data
# raw OSM substation data
path_raw_substations = 'data/raw/africa_all_raw_substations.geojson'
# raw OSM lines data
path_raw_lines = 'data/raw/africa_all_raw_lines.geojson'

# network paths
path_cluster_network = 'networks/elec_s_100.nc'
# solve network output
solved_network_path = 'results/networks/elec_s_100_ec_lcopt_Co2L-3H.nc'

# Images setups
max_width_image = 20
max_height_image = 20
dpi=300

### Python imports

In [ ]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import requests
import pypsa
import shutil
from rasterio.plot import show
from shapely.wkt import loads
from shapely.geometry import Point
from shapely.geometry import LineString

import os
import sys
sys.path.append('../')  # to import helpers
from scripts._helpers import sets_path_to_root
sets_path_to_root("pypsa-africa")
sys.path.append('./scripts')  # to import helpers
from download_osm_data import create_country_list

### Auxiliary functions

In [ ]:
def calculate_width_height_image(width_image,
                           height_image,
                           max_width=max_width_image,
                           max_height=max_height_image):
    """
    Function to identify the width and height of an image to plot
    while keeping the proportions of the image
    """
    if width_image/height_image >= max_width/max_height:
        # image width is the limiting factor
        return (width_image, max_width/width_image*height_image)
    else:
        # image height is the limiting factor
        return (max_height/height_image*width_image, max_height)

### File imports

In [ ]:
# gadm file imports
countries = gpd.read_file(path_country_shapes)
off_shore = gpd.read_file(path_off_shore_shapes)
gadm = gpd.read_file(path_gadm_shapes)

# OSM data imports
df_substations_osm_raw = gpd.read_file(path_raw_substations)
df_lines_osm_raw = gpd.read_file(path_raw_lines)

# network imports
# cluster network
n_c = pypsa.Network(path_cluster_network)
# solved network
n_solve = pypsa.Network(solved_network_path)

## Continent and raw OSM data

### Plot for the entire area downloaded

In [ ]:
total_bounds_countries = countries.total_bounds
delta_bounds_xy = (total_bounds_countries[2] - total_bounds_countries[0],  # maxx - minx
                    total_bounds_countries[3] - total_bounds_countries[1])  # maxy - miny

size_image = calculate_width_height_image(*delta_bounds_xy)

ax2= gadm.plot(
    column="pop",
    cmap='OrRd',
    figsize=size_image,
    legend=None,
    norm=matplotlib.colors.LogNorm(vmin=gadm["pop"].min()+1,
    vmax=gadm["pop"].max(),clip=True)) #column="pop", 
off_shore.plot(ax=ax2, label="offshore")
df_lines_osm_raw.plot(ax=ax2, color="navy")
df_substations_osm_raw.plot(ax=ax2, color="papayawhip", alpha=0.7, markersize = 3)

plt.axis("off")
plt.savefig("continent_image.png", dpi=dpi)

### Plot for a subregion specified by a string as in "countries" config

In [ ]:
region = ["africa"]

list_countries = create_country_list(region)

ax2= gadm[gadm.country.isin(list_countries)].plot(
    column="pop",
    cmap='OrRd',
    figsize=size_image,
    legend=None,
    norm=matplotlib.colors.LogNorm(vmin=gadm["pop"].min()+1,
    vmax=gadm["pop"].max(),clip=True)) #column="pop", 
off_shore[off_shore.name.isin(list_countries)].plot(ax=ax2, label="offshore")
df_lines_osm_raw[df_lines_osm_raw.Country.isin(list_countries)].plot(ax=ax2, color="navy")
df_substations_osm_raw[df_substations_osm_raw.Country.isin(list_countries)].plot(
    ax=ax2, color="papayawhip", alpha=0.7, markersize = 3)

plt.axis("off")
plt.savefig("region_image.png", dpi=dpi)

### Plot country shape and OSM data

In [ ]:
ax = countries.boundary.plot(figsize=(10,10))
# off_shore.plot(ax=ax)
df_substations_osm_raw.plot(ax=ax, color="blue")
df_lines_osm_raw.plot(ax=ax, color="orange")
plt.axis('off')
plt.savefig("border_osm_total.png", dpi=dpi)

### Plot country shape and OSM data for the region

In [ ]:
ax = countries[countries.name.isin(list_countries)].boundary.plot(figsize=(10,10))
# off_shore.plot(ax=ax)
df_substations_osm_raw[df_substations_osm_raw.Country.isin(list_countries)].plot(ax=ax, color="blue")
df_lines_osm_raw[df_lines_osm_raw.Country.isin(list_countries)].plot(ax=ax, color="orange")
plt.axis('off')
plt.savefig("border_osm_region.png", dpi=dpi)

### Plot country shape and OSM data for a specific country

In [ ]:
specific_country = ["NG"]

ax = gadm[gadm.country.isin(specific_country)].boundary.plot(figsize=(4,4))
# off_shore.plot(ax=ax)
df_substations_osm_raw[df_substations_osm_raw.Country.isin(specific_country)].plot(ax=ax, color="blue")
df_lines_osm_raw[df_lines_osm_raw.Country.isin(specific_country)].plot(ax=ax, color="orange")
plt.axis('off')
plt.savefig("border_osm_specificregion.png", dpi=dpi)

## Cluster network features

### Plot cluster network

In [ ]:
# buses dataframe
buses_c = n_c.buses
buses_c["geometry"] = gpd.points_from_xy(buses_c.x, buses_c.y)
buses_c = gpd.GeoDataFrame(buses_c, crs='epsg:4326')

# lines dataframe
lines_c = n_c.lines
lines_c['geometry'] = lines_c.apply(lambda x: LineString([buses_c.loc[x["bus0"], "geometry"], buses_c.loc[x["bus1"], "geometry"]]), axis=1)
lines_c = gpd.GeoDataFrame(lines_c, crs='epsg:4326')

n_c.plot(bus_colors="blue", line_colors="orange")

plt.savefig("cluster_network.png", dpi=dpi)

## Data report on installed generation

In [ ]:
# filter only generators
generators = n_solve.generators

# obtain installed generation by carrier
generators.groupby(by='carrier')['p_nom_opt'].sum()

In [ ]:
n_solve.links.groupby(by='carrier')['p_nom_opt'].sum()

In [ ]:
n_solve.storage_units.groupby(by='carrier')['p_nom_opt'].sum()/1000

In [ ]:
n_solve.storage_units